# <font face="BIZ UDゴシック">スレッドと排他制御</font>


## キーポイント

*
*

書くことリスト

* プロセスとスレッドの違い
* スレッドID
* スレッドが切り替えるデータ
* スレッドローカルストレージ
* thread
* jthread
* yield
* sleep_for
* stop_token
* stop_source
* async
* future
* mutex
* lock_guard
* condition_vairable
* atomic


---

## <font face="BIZ UDゴシック">1 スレッド</font>

---


### 1.1 プロセスとスレッド

$
\boxed{ \qquad \qquad \qquad \qquad \quad プロセス \\
\boxed{メインスレッド} ~ \boxed{サブスレッド1} ~ \boxed{サブスレッド2} ~ …
}
$

プログラムが実行されると、OS(オペレーティング・システム)によって、「プロセス」と呼ばれる「プログラム全体を管理するためのデータ構造」が作られます。その次に、「スレッド」と呼ばれる「実際にプログラムを実行する構造」が作られます。

スレッドには、プロセスに含まれる任意の関数を指定して実行させることができます。OSは最初に作成するスレッドに`main`関数を割り当てることで、プログラムの実行を開始します。

この「プロセスの最初に作られたスレッド」のことを、「メイン・スレッド」または「プライマリ・スレッド」と呼びます。<br>
プロセスは複数のスレッドを持つことができます。

$
[スレッドの状態の変化] \\
\quad \boxed{
\boxed{　待機状態　} \\
~~~~~~~ ↓ \space ↑ \\
\boxed{　実行状態　} \\
~~~~~~~~~ ↓ \\
\boxed{　終了状態　}} \\
$

各スレッドは作成されると「待機状態」になり、OSから指示で「実行状態」になります。<br>
そして、スレッドが終了すると「終了状態」になります。

また、スレッド自身あるいはOSからの指示で、一時的に「待機状態」に戻ることがあります。

これらの状態のうち、実際にプログラムを実行するのは「実行状態」だけです。<br>
「待機状態」と「終了状態」では、プログラムは1文字も実行されません。


### 1.2 thread(スレッド)クラス

C++言語でスレッドを作成するには`thread`(スレッド)クラスを使います。<br>
`thread`クラスには、基本的な機能を持つスレッドを作成し、管理するための機能が備えられています。

`thread`クラスは次のように定義されています。

>注意: あまり使わないメンバは省略して書いています。正確な定義は以下のサイト等を参照してください。<br>
>`https://cpprefjp.github.io/reference/thread/thread.html`

```cpp
class thread
{
public:
  class id {}; // スレッドIDをあらわすクラス(詳細は省略)

  // デフォルトコンストラクタ
  thread();

  // 関数と引数を指定するコンストラクタ
  template<typename F, typename... A>
  thread(F&& f, A&&... a);

  // コピー代入を禁止
  thread(const thread&) = delete;
  thread& operator=(const thread&) = delete;

  // デストラクタ
  ~thread();

  // メンバ関数
  void join();       // ジョイン関数
  void detach();     // デタッチ関数
  id get_id() const; // ゲット・アイディ関数
};
```

`thread`クラスを使うには`thread`という名前のヘッダファイルをインクルードします。

`thread`クラスの使い方は次のとおりです。

1. コンストラクタに実行させたい関数を指定して、変数を作成する(上記の２つ目の`template`コンストラクタ)
2. 適当なタイミングで`join`(ジョイン)メンバ関数を呼び出し、スレッドの終了を待つ

以下のプログラムは、`thread`クラスを使った例です。

**コード**

```cpp
#include <iostream>
#include <string>
#include <thread>
using namespace std;

int func(int n) {
  string s = string("スレッド") + to_string(n) + "を実行\n";
  cout << s;
  return n;
}

int main() {
  cout << "プログラム開始" << endl;

  thread t1(func, 1); // スレッドを作成
  thread t2(func, 2); // もうひとつスレッドを作成

  t1.join(); // スレッドの完了を待つ
  t2.join(); // もう一つのスレッドの完了を待つ

  cout << "プログラム終了" << endl;
}
```

**実行結果**

```txt
プログラム開始
スレッド1を実行
スレッド2を実行
プログラム終了
```

コンストラクタは、次のように引数を指定して呼び出します。

```cpp
thread 変数名(実行させる関数名, 関数の引数1, 関数の引数2, ...);
```

引数を持たない関数を実行させる場合は、最初の関数名だけを指定します。

`join`メンバ関数は、指定のスレッドが終了するまで、メインスレッドの実行を一時停止します。スレッドが終わるまで待っているイメージです。

```cpp
変数名.join();
```

スレッドが処理したデータを使う直前に`join`を呼び出すことで、確実に処理済みのデータを利用できます。<br>
なお、もし`join`を呼び出したときにスレッドが既に完了している場合は、一時停止は行われずにプログラムの実行を続けます。

スレッドが処理したデータをすぐには使わない場合は、`join`の代わりに`detach`(デタッチ)メンバ関数を使います。

```cpp
変数名.detach();
```

`thread`型の変数を削除するより前に、「必ず`join`の`detach`どちらかひとつを実行する」必要がある点に注意して下さい。<br>
呼び出しを忘れたり、2回以上呼び出してしまうと実行時エラーが発生します。

>**【すぐ終わる関数にスレッドを使わないこと】**<br>
>上の例の場合、関数`func`を直接呼び出すほうが簡単で、しかも高速です。<br>
>短い関数の場合、スレッド作成にかかる時間のほうが長くなってしまうためです。<br>
>しかし、ファイル読み込みのような「非常に時間のかかる処理」をする場合、スレッドを使うことで実行時間を短くできる可能性があります。


### 1.3 スレッドを作成する目的

スレッドを作成する目的には、以下のものがあります。

* 応答性の改善
* 実行速度の向上

「応答性の改善」は、例えば「ファイルの読み込み中でもUIが反応するように、読み込みスレッドとUIスレッドを分ける」方法が挙げられます。読み込み中でもUIスレッドは入力に反応できるので、「読み込み中はクリックしても反応しない」といった状況を避けられます。

この例では、2つのスレッドは独立して動作し、影響し合うことはあまりありません。

「実行速度の向上」は、例えば「ファイルをコピーするとき、読み込みスレッドと書き込みスレッドを分ける」ことが挙げられます。すべてのデータを読み込んでから書き込むのではなく、読み込みを続けながら、読み込み済みの部分を同時に書き込むことで、処理時間を短くできます。

この例では、読み込みスレッドがファイルを一定量読み込んだら、書き込みスレッドに「ここまで書き込んでほしい」という指示を出す必要があります。

指示を出すには、スレッドの間で情報をやり取りする、つまりスレッド間通信ができなくてはなりません。


### 1.4 スレッドの実行を制御する

複数のスレッドがあるとき、それらのスレッドは可能な限り同時に実行されます。しかし、ときには「他のスレッドが処理を終えるまでしばらく待ちたい」とか、「自分より他のスレッドを優先して実行させたい」ということがあります。

C++には、スレッドの実行を制御するために以下の機能が用意されています。

* `sleep_for`(スリープ・フォー): 指定された時間だけ、スレッドを一時停止する
* `sleep_until`(スリープ・アンティル): 指定された時間になるまで、スレッドを一時停止する
* `yield`(イールド): 現在のスレッドを一時停止し、他のスレッドに実行権を渡す

これらの関数は`this_thread`(ジス・スレッド)名前空間のメンバです。<br>
使用するときは`this_thread::yield()`のように書く必要があります。

2025年現在のCPUは、8～32個のスレッドを同時に実行できます。<br>
しかし、この全てをひとつアプリケーションで使えるわけではないのです。なぜなら、OSが実行する必要のある様々なプログラムも、それぞれがスレッドを持っているからです。

それ以前に、実行環境によって同時実行できるスレッド数は大きく変わります。OSは、一定時間以上経過したスレッドを「強制的に一時停止」して、他のスレッドが実行できるようにしてくれますが、これは安全対策に過ぎません。

複数のスレッドを円滑に動作させるには、実行するべきスレッドを適切に決定しなくてはなりません。適切にスレッドを制御しないと「自分のPCではうまく動作するのに、友達のPCでは動きがおかしい」ということが起こり得ます。

>自分のPCで同時に実行できるスレッド数を調べるには、`Ctrl+Shift+Esc`を押して「タスクマネージャー」を開き、「パフォーマンス」タブのCPU情報を見ます。下の方に書いてある「論理プロセッサ数」が、同時に実行できるスレッド数です。


#### sleep_for(スリープ・フォー)関数

`sleep_for`関数は、「指定された時間だけスレッドを一時停止」させます。

```cpp
namespace this_thread {
void sleep_for(一時停止する時間);
}
```

「一時停止する時間」は、`chrono`(クロノ)ライブラリの関数を使って指定します。

| 指定する単位 | 関数名 |
|:-------------|:-------|
| 秒           | <font size=3>chrono::seconds</font>(クロノ・セコンズ) |
| ミリ秒(千分の1秒) | <font size=3>chrono::miliseconds</font>(クロノ・ミリ・セコンズ) |
| マイクロ秒(百万分の1秒) | <font size=3>chrono::microseconds</font>(クロノ・マイクロ・セコンズ) |

>分、時、日などの単位もありますが、スレッドで使うことはまずないので省略しています。詳しくは以下のサイトを参照してください。<br>
>`https://cpprefjp.github.io/reference/chrono.html`

例えば、「2秒間だけ一時停止させたい」場合は次のように書きます。

>以下の`▶`ボタンを押してファイルを保存し、実行してみてください。


In [ ]:
%%writefile sleep_for_sample.cpp
#include <iostream>
#include <thread>
#include <chrono>
using namespace std;

int main() {
  cout << "プログラム開始" << endl;

  for (int i = 1; i <= 5; i++) {
    this_thread::sleep_for(chrono::seconds(2)); // 2秒待つ
    cout << i * 2 << "秒経過" << endl;
  }

  cout << "プログラム終了" << endl;
}

In [ ]:
# @title 実行
!g++ -Wall -Wextra -O2 -o sleep_for_sample sleep_for_sample.cpp && ./sleep_for_sample

`sleep_for`は、実際には「指定された時間 **以上** 待つ」ように設計されています。OSは可能な限り時間を合わせようとしてくれますが、正確性には期待しないで下さい。

基本的には、一時停止の時間が短くなるほど正確性が下がります。「2秒を指定したら2.1秒後に再開した」ならいいほうで、「0.01秒を指定したら0.05秒後に再開した」ということもありえます。


#### sleep_until(スリープ・アンティル)関数

`sleep_until`関数は、指定した時間になるまでスレッドを一時停止させます。「プログラム開始から10秒間隔で特定の処理を実行したい」というような場合に使います。

>`until`は「～まで」という意味の英単語です。

```cpp
namespace this_thread {
void sleep_until(一時停止を終える時刻);
}
```

例えば、`sleep_for`を使ったループでは、`sleep_for`を実行するたびにすこしずつ時間のズレが溜まっていきます。しかし、代わりに`sleep_until`を使えば「開始からN秒後」という指定ができるため、時間のズレが溜まっていくことはありません。

「現在の時刻」を取得するには、`chrono`ライブラリの`system_clock::now`(システムクロック・ナウ)関数を使います。

例えば、「2秒間隔で一時停止させたい」場合は次のように書きます。

>以下の▶ボタンを押してファイルを保存し、実行してみてください。


In [ ]:
%%writefile sleep_until_sample.cpp
#include <iostream>
#include <thread>
#include <chrono>
using namespace std;

int main() {
  cout << "プログラム開始" << endl;

  auto start = chrono::system_clock::now(); // 現在の時刻を得る
  for (int i = 1; i <= 5; i++) {
    this_thread::sleep_until(start + chrono::seconds(i * 2)); // 2秒間隔で待つ
    cout << i * 2 << "秒経過" << endl;
  }

  cout << "プログラム終了" << endl;
}

In [ ]:
# @title 実行
!g++ -Wall -Wextra -O2 -o sleep_until_sample sleep_until_sample.cpp && ./sleep_until_sample

`sleep_until`関数の正確性自体は`sleep_for`と同等で、つまりあまり当てになりません。

また、ゲームなどでは一時停止中だからといって何もしないわけではない(アニメーションさせたり、ダメージなどに反応する必要がある)ので、`sleep_for`や`sleep_until`の出番は少ないです。


#### yield(イールド)

`yield`関数は、自分のほかに「実行されるのを待っているスレッド」が存在する場合、自分のスレッドの実行権を明け渡します。

待っているスレッドが存在しない場合、この関数は何もしません。

```cpp
namespace this_thread {
void yield();
}
```

コンピューターが同時に実行できるスレッド数は有限なので、OSは一定時間以上実行しているスレッドを強制定期に待機状態に戻し、他の待機状態にあるスレッドを実行状態にします。しかし、この仕組みは安全装置であり、基本的にはスレッド自身が定期的に待機状態に戻ることが推奨されます。`yield`関数はその方法のひとつです。

>**【yieldより排他制御を使おう】**<br>
>`yield`が実行権を明け渡すとは「保証されていない」点に注意して下さい。<br>
>実行スレッドを切り替える場合は、次章で説明する「排他制御」を使うほうが確実です。


### 1.6 future(フューチャー)ライブラリ

`thread`クラスのコンストラクタに指定した関数について、基本的には戻り値は無視されます。<br>
戻り値を取得するには、直接`thread`クラスを使う代わりに、`async`(エイ・シンク)関数と`future`(フューチャー)クラスとを使います。


#### futureクラス

`future`クラスは`async`関数の戻り値で、スレッドに渡した関数の戻り値を取り出す機能を持ちます。<br>
`future`クラスは、次のように定義されています。<br>
テンプレートパラメータ`R`は、「関数の戻り型」になります。

```cpp
template<typename R>
class future
{
public:
  // コンストラクタ・デストラクタ
  future();
  ~future();

  // コピー代入を禁止
  future(const future&) = delete;
  future& operator=(const future&) = delete;

  // 戻り値を返す
  R get();
};
```

関数の戻り値は、`get`メンバ関数を使うことで取得できます。<br>
もし`async`で作成したスレッドがまだ完了していない場合、`get`を呼び出したスレッドを「待機状態」にします。<br>
そして、`async`で作成したスレッドが完了すると、自動的に「実行状態」に戻ります。


#### async関数

`async`関数は、スレッドを作成して`future`型のオブジェクトを返します。

```cpp
template<typename F, typename... A>
future async(launch p, F func, A... args);
```

第１引数には、以下の`launch`(ロンチ)列挙型の値を指定します。

```cpp
enum class launch {
  async,    // スレッドはすぐに実行される
  deferred, // スレッドは戻り値を求められた時点で実行される
};
```

通常は`launch::async`(ロンチ・エイシンク)を指定します。<br>
`launch::deferred`(ロンチ・ディファード)は、スレッドの実行順を制御したい場合に指定しますが、あまり使うことはないでしょう。

第２引数は「スレッドで実行する関数」です。<br>
第３引数以降は「関数に渡す引数」です。

`async`関数や`future`クラスを使うには、`future`という名前のヘッダファイルをインクルードします。<br>
次のプログラムは、`async`関数と`future`クラスを使った例です。

**コード**

```cpp
#include <iostream>
#include <future>
#include <vector>
using namespace std;

vector<int> func()
{
  vector<int> v(10);
  for (int i = 0; i < 10; i++) {
    v[i] = i;
  }
  return v;
}

int main()
{
  future<vector<int>> f = async(launch::async, func); // スレッドを作成し、futureを取得

  vector<int> v = f.get(); // スレッドの完了を待って、戻り値を取得

  // 取得した戻り値を出力
  for (int i = 0; i < (int)v.size(); i++) {
    cout << v[i] << ' ';
  }
  cout << endl;
}
```

**実行結果**

```txt
0 1 2 3 4 5 6 7 8 9
```

このように、`async`と`future`は、「スレッドの実行結果をまとめて受け取れたら十分」という場合に便利です。

しかし、「実行結果を少しずつ受け取りたい」とか、「スレッドにデータの読み込みを任せつつ、読み込み完了率を表示する」という場合には機能不足です。そのような用途には、次の章で説明する「排他制御(はいた せいぎょ)」を使うほうがよいでしょう。


---

## <font face="BIZ UDゴシック">2 排他制御(はいた せいぎょ)</font>

---


### 2.1 スレッド間通信、データ競合、排他制御

複数のスレッドの間でデータをやり取りすることを「スレッド間通信」といいます。<br>
スレッド間通信の基本的な考え方は、

&emsp;**グローバル変数や参照、ポインタを使って、どのスレッドからでもデータを読み書き可能にする**

ことです。

問題は、「普通の変数を使うだけでは通信に失敗する」ことです。<br>
というのも、２つのスレッドは同時に実行されるため、次のような状況が起こり得るのです。

* 一方がデータを書き込んでいる途中で、他方が不完全なデータを読み取ってしまう
* 互いに同時にデータを書き込んでしまう

これらの状況のことを「データ競合」といいます。<br>
データ競合が起こると、プログラムは意図したとおりに動作できなくなってしまいます。

データ競合を防ぐには、「あるスレッドがデータを読み書きしている間、他のスレッドの読み書きを禁止する仕組み」が必要になります。この仕組みのことを「排他制御(はいた せいぎょ)」といいます。

C++では、排他制御の手段として以下の方法が用意されています。

* `atomic`(アトミック)
* `mutex`(ミューテックス)
* `lock_guard`(ロック・ガード)
* `unique_lock`(ユニーク・ロック)
* `condition_variable`(コンディション・バリアブル、「状態変数」という意味)


### 2.2 atomic(アトミック)クラス

`atomic`クラスは、データに対して排他制御を追加します。排他制御によって、データ競合のない読み書きが可能になります。

>`atomic`は「不可分な」という意味です。プログラミング言語では、データの読み込みや書き込みが「他のスレッドに邪魔されることなく完了する」という意味で使われます。

`atomic`クラスは次のように、クラステンプレートとして宣言されています。

```cpp
template<typename T>
class atomic;
```

`atomic`クラスを使うには`atomic`という名前のヘッダファイルをインクルードします。

使い方は、テンプレートパラメータ`T`に排他制御したい型を指定して、変数を作成するだけです。`atomic`を使って宣言された変数は、「アトミック変数」と呼ばれます。

```cpp
#include <atomic>
#include <string>
using namespace std;

atomic<int> i;    // int型のアトミック変数
atomic<double> d; // double型のアトミック変数
atomic<string> s; // string型のアトミック変数

struct A { int x, y, z; };
atomic<A> a; // 構造体 A 型のアトミック変数
```

アトミック変数の読み書きは、基本的には普通の変数と同じです。代入や四則演算もできますし、他の変数と混ぜても構いません。<br>
ただし、複合代入演算子には制限があり、使えるのは`+=`(加算代入)と`-=`(減算代入)だけです。`*=`(乗算代入)、`/=`(除算代入)、`%=`(剰余代入)を書くとコンパイルエラーになります。

&emsp;<font color=#4e6>OK</font>: `+=`, `-=`<br>
&emsp;<font color=#e46>NG</font>: `*=`, `/=`, `%=`

次のプログラムは、アトミック変数を使った例です。

**コード**

```cpp
#include <iostream>
#include <atomic>
using namespace std;

// アトミック変数
atomic<int> a;

int main() {
  a = 1;
  a += 2;
  a--;       // OK. 「a -= 1」と同じ
  // a *= 2; // NG. 乗算代入はできない
  cout << a << endl;

  // 普通の変数を組み合わせて式を作ることもできる
  int n = 5;
  a -= n;
  cout << a * 2 << endl;

  // 比較もできる
  atomic<int> b = 57;
  if (a == b) {
    cout << "a と b は等しい" << endl;
  } else {
    cout << "a と b は異なる" << endl;
  }
}
```

**実行結果**

```txt
2
-6
a と b は異なる
```


#### スレッドとアトミック変数

アトミック変数を使うと、データ競合を起こすことなくスレッドを停止させたり、処理を分岐することができます。

この用途には、`atomic`クラスの`wait`(ウェイト)メンバ変数と、`notify_all`(ノーティファイ・オール)メンバ関数を使います。

```cpp
void wait(待機する値);
void notify_all();
```

`wait`メンバ関数は、アトミック変数の値が「待機する値 **以外** 」に変更されるまで、自分のスレッドを待機状態にします。自分は待機状態になるので、値の変更は他のスレッドで行う必要があります。

`notify_all`メンバ関数は、`wait`によって待機状態になったスレッドを実行状態に戻します。`notify_all`を呼び出す前にアトミック変数の値を変更しておく必要があります。値の変更が行われないと、せっかく実行状態に戻してもすぐに待機状態に戻ってしまうからです。

>`notify`は「通知する」という意味です。`notify_all`の場合、他のスレッドに「アトミック変数の値をチェックしてほしい」という通知を出すイメージで使われています。


In [ ]:
%%writefile atomic_sample.cpp
#include <iostream>
#include <thread>
#include <atomic>
using namespace std;

atomic<int> value; // 表示用のアトミック変数
atomic<bool> flag; // 待機用のアトミック変数

int main() {
  cout << "プログラム開始" << endl;

  // 関数にはラムダ式も使える
  thread t([]() {
    for (int i = 0; i < 5; i++) {
      flag = false;     // 「待機用のアトミック変数」を false に設定
      flag.wait(false); // 変数の値が false 以外に変更されるまで待機
      cout << value << endl; // 表示用のアトミック変数を出力
    }
  });

  for (int i = 0; i < 5; i++) {
    this_thread::sleep_for(chrono::seconds(1)); // 1秒待つ
    value = i + 1;     // 表示する値を設定
    flag = true;       // 「待機用のアトミック変数」を true に変更
    flag.notify_all(); // 変更を他のスレッドに通知
  }
  t.join();
  cout << "プログラム終了" << endl;
}

In [ ]:
# @title 実行
!g++ -std=c++20 -Wall -Wextra -O2 -o atomic_sample atomic_sample.cpp && ./atomic_sample

このように、基本的なスレッド間通信には、`atomic`クラスが便利です。
<br>
アトミック変数を使うことで、比較的簡単に、複数のスレッドの間でデータをやりとりしたり、条件が整うまでスレッドの実行を待たせることができるからです。



#### atomicクラスに向いていない操作

`atomic`クラスは便利ですが、「すべてのデータを一括で読み書きできなくてはならない」という制限があります。<br>
そのため、`vector`や`unordered_map`などの高機能なクラスを`atomic`にしようとすると、コンパイルエラーになります。<br>
ただし、「コンテナクラスの要素を`atomic`にすることは可能」です。

&emsp;<font color=#4e8>OK</font>: `vector<atomic<int>>`<br>
&emsp;<font color=#e48>NG</font>: `atomic<vector<int>>`

ただし、OKなほうの「`atomic`の配列」は、個々の要素がアトミックなだけです。<br>
そのため、「複数の要素を同時に排他制御しながら読み書きする」ことはできません。

&emsp;<font color=#4e8>OK</font>: 3番目の要素を排他的に読み書き<br>
&emsp;<font color=#e48>NG</font>: 3番目と4番目の要素を同時に排他的に読み書き

次のプログラムは、アトミック要素の配列を、複数書き換える場合に起こる問題の例です。

```cpp
#include <vector>
#include <atomic>
using namespace std;

vector<atomic<int>> a(5);

int main() {
  a[3] = 123; // OK. これは排他制御される
              // NG. ここで他のスレッドに割り込まれる可能性がある
  a[4] = 456; // OK. これも排他制御される
}
```

`atomic`クラスは「簡単に使えること」を重視しているため、複雑な処理には向いていないのです。<br>
複雑な操作をしたい場合は、`atomic`以外の方法を使う必要があります。


### 2.3 mutex(ミューテックス)クラス

`atomic`クラスでは対応できないような「複雑な操作」をしたい場合は、`mutex`(ミューテックス)クラスを使います。<br>
`mutex`クラスは`atomic`クラスより少しだけ扱いが難しいのですが、その代わりに、より高度な操作が可能です。

>`mutex`という単語は、`mutual exclusion`(ミュータル・エクスクルージョン、「排他制御」という意味)の省略形です。

`mutex`クラスは次のように定義されています。

```cpp
class mutex
{
public:
  // コンストラクタ
  mutex();

  // デストラクタ
  ~mutex();

  // コピー代入を禁止
  mutex(const mutex&) = delete;
  mutex& operator=(const mutex&) = delete;

  // メンバ関数
  void lock();
  bool try_lock();
  void unlock();
};
```

`mutex`クラスは「排他制御のための基本機能」を提供します。<br>
基本的な使い方は次のとおりです。

1. `lock`(ロック)または`try_lock`(トライ・ロック)メンバ関数で排他制御を開始
2. `unlock`(アンロック)メンバ関数で排他制御を終了

`mutex`クラスのロックに成功するのは、ひとつのスレッドだけです。<br>
複数のスレッドが`lock`を呼び出すと、そのうちひとつのスレッドだけが`lock`メンバ関数から戻ります。<br>
残りのスレッドは`lock`メンバ関数の内部で「待機状態」になります。

`lock`から戻ったスレッドが`unlock`を呼び出すと、ロックが解除されます。そして、次にOSがスレッドの実行チェックを行うときに、「待機状態」のスレッドのひとつが「実行状態」になり、`lock`メンバ関数から戻ります。

&emsp;&emsp;[`lock`と`unlock`の動作の概念図]

$
\boxed{\boxed{スレッドA}}→\boxed{lock}→データを操作→\boxed{unlock}→→ \\
　　　　　　　　　　　　　　　　　　　　　　⇣ \\
\boxed{\boxed{スレッドB}}→→\boxed{lock…(待機状態)…(実行状態)　　　　　}→データを操作→\boxed{unlock}→ \\
$

排他制御したいデータは、「このデータは`lock`と`unlock`の間でのみ読み書きする」と決めます。<br>
これはC++言語の機能ではなく、「アプリを書くうえで守るべきルール」です。<br>
このときよく使われる方法は、このルールを強制するクラスを定義して、アプリでは常にそのクラスを使うことです。

`mutex`クラスを使うには、`mutex`という名前のヘッダファイルをインクルードします。<br>
次のプログラムは、`mutex`クラスを使った例です。

**コード**

```cpp
#include <iostream>
#include <vector>
#include <thread>
#include <mutex>
#include <chrono>
using namespace std;

mutex m;
vector<int> v;

void func(int n)
{
  for (int i = 0; i < 5; i++) {
    this_thread::sleep_for(chrono::milliseconds(1)); // 他のスレッドに切り替わるように少し待機
    m.lock();                   // ミューテックスをロック
    v.push_back(n + i * 2);     // データ操作
    v.push_back(n + i * 2 + 1); // データ操作
    m.unlock();                 // ミューテックスをアンロック
  }
}

int main() {
  thread t1(func, 0);  // スレッド1
  thread t2(func, 10); // スレッド2
  t1.join();
  t2.join();

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << ' ';
  }
  cout << endl;
}
```

**実行結果**

```txt
0 1 10 11 2 3 12 13 4 5 14 15 6 7 16 17 8 9 18 19  
```

この実行結果には、1桁の数値と2桁の数値が混ざっています。1桁の数値はスレッド`t1`が追加したもので、2桁の数値はスレッド`t2`が追加したものです。

1桁の数値も2桁の数値も、必ず2個ずつ連続している点に注目してください。<br>
これは、2回の`push_back`呼び出しを、ミューテックスの`lock`と`unlock`の間で実行しているためです。

なお、スレッドが実行される順番はコンピューター次第です。上記の実行結果は、たまたまスレッド`t1`が先に実行されたに過ぎません。スレッド`t2`が先に実行されたり、どちらかのスレッドが2回連続して実行されることもありえます。


#### lockとunlockの注意点

`mutex`の`lock`と`unlock`メンバ関数は、必ず1対1で使わなくてはなりません。<br>
`lock`していないのに`unlock`を実行したり、既に`lock`している状態でもう一度`lock`を実行してしまうと、実行時エラーになります。

```cpp
mutex m;

void func() {
  //m.unlock(); // NG. ロックしていない

  m.lock();
  //m.lock(); // NG. 既にロックしている
  m.unlock();
}
```

上記の例では、簡単に`lock`と`unlock`の間違った使い方を見つけられます。ですが、もっと長いプログラムにおいて、例えば「ある特定の条件の時だけ`unlock`が実行されずに`lock`が連続で実行される」というような問題を見つけることは、かなり難しいです。

つまり、`mutex`の`lock`と`unlock`には、`new`と`delete`を使う場合と同様の問題があるわけです。


### 2.4 lock_guard(ロック・ガード)クラス

`new`と`delete`を使う場合の問題は、それらを自動化してくれるスマートポインタによって解決できるのでした。<br>
同様に、`lock`と`unlock`についても、自動化してくれるクラスを使えば解決できるはずです。

C++には`lock`と`unlock`を自動化するために、以下のクラスが用意されています。

| クラス名 | 機能 |
|:--------:|:-----|
| <font size=3>lock_guard</font><br>(ロック・ガード) | <font size=3>`lock`と`unlock`を完全に自動化する</font> |
| <font size=3>unique_lock</font><br>(ユニーク・ロック) | <font size=3>`lock`と`unlock`を自動化するだけでなく、手動で制御する機能も備える。</font> |
| <font size=3>scoped_lock</font><br>(スコープド・ロック) | <font size=3>複数の`mutex`をまとめてロックでき、それらは自動化される</font> |

なお、この3つのクラスは、`mutex`ヘッダファイルに含まれます。<br>
そのため、`mutex`クラスを使う場合、これらのクラスも自動的に使えるようになります。

まず、`lock_guard`クラスについて見ていきましょう。<br>
`lock_guard`は、「コンストラクタで`lock`し、デストラクタで`unlock`する」だけのクラスです。<br>


```cpp
template<typename M>
class lock_guard
{
public:
   // コンストラクタ
  lock_guard(M& mutex);

  // デストラクタ
  ~lock_guard();

  // コピー代入を禁止
  lock_guard(const lock_guard&) = delete;
  lock_guard& operator=(const lock_guard&) = delete;
};
```

次の例は、`mutex`の`lock`と`unlock`を使ったプログラムを、`lock_guard`クラスで置き換えたものです。

**コード**

```cpp
#include <iostream>
#include <vector>
#include <thread>
#include <mutex>
#include <chrono>
using namespace std;

mutex m;
vector<int> v;

void func(int n)
{
  for (int i = 0; i < 5; i++) {
    this_thread::sleep_for(chrono::milliseconds(1)); // 他のスレッドに切り替わるように少し待機
    lock_guard lg(m);           // コンストラクタで自動的にロック
    v.push_back(n + i * 2);     // データ操作
    v.push_back(n + i * 2 + 1); // データ操作
    // デストラクタで自動的にアンロック
  }
}

int main() {
  thread t1(func, 0);  // スレッド1
  thread t2(func, 10); // スレッド2
  t1.join();
  t2.join();

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << ' ';
  }
  cout << endl;
}
```

**実行結果**

```txt
0 1 10 11 2 3 12 13 4 5 14 15 6 7 16 17 8 9 18 19  
```

`lock_guard`はスコープの終わりで`mutex`をアンロックします。<br>
アンロックするべきタイミングを越えてスコープが続く場合、「`lock_guard`のためのスコープ」を追加する必要があるかもしれません。

>`lock_guard`クラスはほとんどの場面で最初に選ぶべき排他制御です。<br>
>排他制御が必要な場面では、とりあえず`lock_guard`から始めてみるとよいでしょう。


### 2.5 unique_lock(ユニーク・ロック)クラス

`unique_lock`クラスは、`lock_guard`クラスと同様に「コンストラクタで`lock`し、デストラクタで`unlock`する」機能を持っています。<br>
それだけでなく、手動で`lock`や`unlock`を実行する機能も持っています。

`unique_lock`クラスは次のように定義されています。<br>
テンプレートパラメータ`M`はミューテックスの種類です。通常は気にしなくて構いません。

```cpp
template<typename M>
class unique_lock
{
public:
  // コンストラクタ・デストラクタ
  unique_lock(M& mutex);
  unique_lock(M& mutex, defer_lock_t);
  ~unique_lock();

  // コピー代入を禁止する
  unique_lock(const unique_lock&) = delete;
  unique_lock& operator=(const unique_lock&) = delete;

  // メンバ関数
  void lock();
  bool try_lock();
  void unlock();
};
```

`unique_lock`クラスは、次のような場合に使います。

* 条件によって、ロックやアンロックのタイミングが異なる場合がある
* ロックできない場合は待機状態に入らず、他の処理を続けたい

上記のような場合でない限り、`lock_guard`を使うべきです。


### 2.6 scoped_lock(スコープド・ロック)クラス

複数のデータが個別に`mutex`で排他制御されている状況を考えます。<br>
そして、プログラムの大半では、データを個別に排他制御するだけで済むとします。<br>
しかし、2つ以上のデータを関連付けて処理するときは、一時的に複数のデータをまとめて排他制御する必要があります。

これは、複数の`mutex`をひとつずつロックすることで対応できます。ただし、ロックとアンロックの順序を適切に指定し、また常に同じ順序で書かなくてはなりません。<br>
もし間違えると「デッド・ロック」という問題が発生します。

`scoped_lock`クラスを使うと、デッドロックを起こすことなく、複数の`mutex`を管理できます。

`scoped_lock`クラスは次のように定義されています。<br>
テンプレートパラメータ`M`はミューテックスの種類です。通常は気にしなくて構いません。

```cpp
template<typename... M>
class scoped_lock
{
public:
  // コンストラクタ・デストラクタ
  scoped_lock(M&... mutex);
  ~scoped_lock();

  // コピー代入を禁止する
  scoped_lock(const scoped_lock&) = delete;
  scoped_lock& operator=(const scoped_lock&) = delete;
};
```

`lock_guard`クラスと同じく、`scoped_lock`クラスは「コンストラクタで全てのミューテックスをロックし、デストラクタで全てアンロックする」機能だけを持ちます。

`scoped_lock`が役に立つのは、「複数のデータを同時に排他制御したい」という場合に限られます。<br>
それ以外の場合は、基本的に`lock_guard`クラスを使うべきです。

**コード**

```cpp
```



